In [1]:
import numpy as np
import cv2 as cv

In [3]:
BLUE, GREEN, RED, BLACK, WHITE = (255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 0, 0), (255, 255, 255)

DRAW_BG = {'color' : BLACK, 'val' : 0}
DRAW_FG = {'color' : WHITE, 'val' : 1}

In [4]:
rect = (0, 0, 1, 1)

drawing = False
rectangle = False
rect_over = False
rect_or_mask = 100
value = DRAW_FG
thickness = 3

In [2]:
def onMouse(event, x, y, flags, param):
    global ix, iy, img, img2, drawing, value, mask, rectangle
    global rect, rect_or_mask, rect_over

    if event == cv.EVENT_RBUTTONDOWN:
        rectangle = True
        ix, iy = x, y
    elif event == cv.EVENT_MOUSEMOVE:
        if rectangle:
            img = img2.copy()
            rect = (min(ix, x), min(iy, y), abs(ix-x), abs(iy-y))
            rect_or_mask = 0
    elif event == cv.EVENT_RBUTTONUP:
        rectangle = False
        rect_over = True
        cv.rectangle(img, (ix, iy), (x,y), RED, 2)
        rect = (min(ix, x), min(iy, y), abs(ix-x), abs(iy-y))
        rect_or_mask = 0
        print('n:적용하기')

    if event == cv.EVENT_LBUTTONDOWN:
        if not rect_over:
            print("마우스 왼쪽 버튼을 누른채로 전경이 되는 부분을 선택하세요")
        else:
            drawing = True

            cv.circle(img, (x, y), thickness, value['color'], -1)
            cv.circle(mask, (x, y), thickness, value['val'], -1)

    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            cv.circle(img, (x, y), thickness, value['color'], -1)
            cv.circle(mask, (x, y), thickness, value['val'], -1)
    elif event == cv.EVENT_LBUTTONUP:
        if drawing:
            drawing = False
            cv.circle(img, (x, y), thickness, value['color'], -1)
            cv.circle(mask, (x, y), thickness, value['val'], -1)

    return


In [ ]:
def grabcut(img_path):
    global ix, iy, img, img2, drawing, value, mask, rectangle
    global rect, rect_or_mask, rect_over

    img = cv.imread(img_path)
    img2 = img.copy

    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    output = np.zeros(img.shape, np.uint8)
    cv.namedWindow('input')
    cv.namedWindow('output')
    cv.setMouseCallback('input', onMouse, param=(img,img2))
    cv.moveWindow('input', img.shape[1] + 10, 90)
    print('오른쪽 마우스 버튼을 누르고 영역을 지정한 후 n을 누르세요')

    while True:
        cv.imshow('output', output)
        cv.imshow('input', img)
        k = cv.waitKey(1) & 0xFF
        if k == 27:
            break
        if k == ord('0'):
            print('왼쪽 마우스로 제거할 부분을 표시한 후 n을 누르세요')
            value = DRAW_BG
        elif k == ord('1'):
            print('왼쪽 마우스로 복원할 부분을 표시한 후 n을 누르세요')
            value = DRAW_FG
        elif k == ord('r'):
            print('리셋합니다.')
            rect = (0, 0, 1, 1)
            drawing = False
            rectangle = False
            rect_or_mask = 100
            rect_over = False
            value = DRAW_FG
            img = img2.copy()
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            output = np.zeros(img.shape, np.uint8)
            print('0: 제거배경 선택, 1: 복원 전경 선택, n: 적용하기, r: 리셋')
        elif k == ord('n'):
            bgdModel = np.zeros((1, 65), np.float64)
            fgdModel = np.zeros((1, 65), np.float64)

            if rect_or_mask == 0:
                cv.grabCut(img2, mask, rect, bgdModel, fgdModel, 1, cv.GC_INIT_WITH_RECT)
                rect_or_mask = 1
            elif rect_or_mask == 1:
                cv.grabCut(img2, mask, rect, bgdModel, fgdModel, 1, cv.GC_INIT_WITH_MASK)
            print('0: 제거배경 선택, 1: 복원 전경 선택, n: 적용하기, r: 리셋')

        mask2 = np.where((mask==1) + (mask==3), 255, 0).astype('uint8')
        output = cv.bitwise_and(img, img, mask=mask2)
        cv.imwrite('cut', img)

    cv.destroyAllWindows()

Error: Session cannot generate requests

In [7]:
grabcut('images.jpg')

오른쪽 마우스 버튼을 누르고 영역을 지정한 후 n을 누르세요


error: OpenCV(4.5.3) D:\Program Files\opencv_source\opencv-4.5.3\modules\imgcodecs\src\loadsave.cpp:732: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'cv::imwrite_'
